<a href="https://colab.research.google.com/github/kk98033/NTTU-Digital-System-Design-Lab-Project/blob/main/llamaindex_ReAct_%2B_CitationQueryEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install milvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 10.7 MB/s eta 0:00:00


In [2]:
!pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# 卸載 grpcio，然後安裝特定版本的 grpcio（1.60.0）安裝其他的版本可能會導致衝突
!pip uninstall grpcio
!pip install grpcio==1.60.0

Found existing installation: grpcio 1.60.0
Uninstalling grpcio-1.60.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/grpc/*
    /usr/local/lib/python3.10/dist-packages/grpcio-1.60.0.dist-info/*
Proceed (Y/n)? y
  Successfully uninstalled grpcio-1.60.0
  Using cached grpcio-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)


In [1]:
import os
from google.colab import userdata

# 設定OpenAI API密鑰
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [2]:
from llama_index.core import (
   SimpleDirectoryReader,
   VectorStoreIndex,
   StorageContext
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.vector_stores.milvus import MilvusVectorStore

In [3]:
from milvus import default_server
# 啟動 Milvus lite 服務。
default_server.stop()
default_server.cleanup()
default_server.start()

In [4]:
!wget 'https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT' -O 原住民資料.pdf

--2024-04-10 15:21:59--  https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT
Resolving drive.google.com (drive.google.com)... 142.250.145.138, 142.250.145.102, 142.250.145.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.145.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download [following]
--2024-04-10 15:21:59--  https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.119.132, 2a00:1450:4013:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.119.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5073207 (4.8M) [application/octet-stream]
Saving to: ‘原住民資料.pdf’

原住民資料.pdf      100%[===================>]   4.84M  26.0MB/s    

In [5]:
# load data
tw_docs = SimpleDirectoryReader(
   input_files=["./原住民資料.pdf"]
).load_data()

In [6]:
# build index
# vector_store_tw = MilvusVectorStore(dim=1536, collection_name="tw", overwrite=True)

# for milvus lite users
# vector_store_tw = MilvusVectorStore(host="localhost", port=default_server.listen_port, dim=1536, collection_name="tw", overwrite=True)
# vector_store = MilvusVectorStore(
#     collection_name="citations",
#     host="127.0.0.1",
#     port=default_server.listen_port,
#     dim=vector_dimension
# )

vector_store = MilvusVectorStore(
    collection_name="citations",
    host="127.0.0.1",
    port=default_server.listen_port,
    dim=1536,
    output_fields=["document_id", "title_vector", "summary_vector"]
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: e9234b5a0a3f482587b3d461fd6c3a13
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: citations
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: citations


In [7]:
storage_context_tw = StorageContext.from_defaults(vector_store=vector_store)
tw_index = VectorStoreIndex.from_documents(tw_docs, storage_context=storage_context_tw)

# persist index
tw_index.storage_context.persist(persist_dir="./storage/tw")

In [8]:
tw_engine = tw_index.as_query_engine(similarity_top_k=3)

In [9]:
tw_citation_engine = CitationQueryEngine.from_args(
    tw_index,
    similarity_top_k=3,
    citation_chunk_size=512,
)

In [28]:
response = tw_citation_engine.query("台灣有哪些原住民族？請你說出每一族的特色")
print(response)

for source in response.source_nodes:
    print(source.node.get_text())

台灣有16個原住民族，包括阿美族、泰雅族、排灣族、布農族、卑南族、魯凱族、鄒族、賽夏族、雅美族、邵族、噶瑪蘭族、太魯閣族、撒奇萊雅族、賽德克族、拉阿魯哇族、卡那卡那富族。每個族群都有獨特的特色和分佈區域[1][2][3][6][7]。
Source 1:
台灣每一個民族都有其獨特而值得驕傲的文化特色，值得珍惜、保存與 發揚。清朝時期漢人大量移民來台，強制的同化政策導致平埔族原住民人的血緣與文化，面臨空前的消失危機，令世人遺憾。今日我們有幸仍保留先民特色的十餘族原住民，未來在台灣文化的發展中仍繼續扮演著重要的角色，也成為台灣最重要的文化資產。   原住民族分佈  引用自	台灣原住民族分佈「台灣族群母語推行委員會」製圖	根據原民會網站的說明： 台灣原住民族約有53萬人，佔總人 口數的2﹪，目前，經政府認定的原住民族有：阿美族、泰雅族、排灣族、布農族、卑南族、魯凱族、鄒族、賽夏族、雅美族、邵族、噶瑪蘭族、太魯閣族、撒奇萊 雅族、賽德克族、拉阿魯哇族、卡那卡那富族等16族。

Source 2:
阿美族：阿美族分佈在中央山脈東側，立霧溪以南，太平洋沿岸的東台縱谷及東海岸平原，大部份居住於平地，只有極少數居於山谷中。 泰雅族：泰雅族分布在台灣中北部山區，包括埔里至花蓮連線以北地區。 排灣族：排灣族以台灣南部為活動區域，北起大武山地，南達恆春，西自隘寮，東到太麻里以南海岸。 布農族：布農族分布於中央山脈海拔一千至二千公尺的山區，廣及於高雄縣那瑪夏鄉、台東縣海端鄉，而以南投縣境為主。 卑南族：卑南族分布於台東縱谷南部。 魯凱族：魯凱族分布於高雄縣茂林鄉、屏東縣霧台鄉及台東縣東興村等地。 鄒族：鄒族主要居住於嘉義縣阿里山鄉，亦分布於南投縣信義鄉。 賽夏族：賽夏族居住於新竹縣與苗栗縣交界的山區，又分為南、北兩大族群。

Source 3:
北賽夏居住於新竹縣五峰鄉，南賽夏居住於苗栗縣南庄鄉與獅潭鄉。

Source 4:
族群概述 台灣位居歐亞大陸與太平洋盆地、親潮與黑潮、漢族與南島民族等多元地理環境、文化與族群的匯流交會地帶。台灣原住民屬於南島語族（Austronesian）的一支，包含原住民各族及與漢文化互相涵化的平埔諸族。不到台灣總人口數百分之二的南島語族，居住活動地分佈達一萬六千餘平方公里，佔全台灣面積百分之四十五。在台灣變異性極大的自然環境中，原住民因應不同的生態而發展出不

In [17]:
query_engine_tools = [
   QueryEngineTool(
       query_engine=tw_citation_engine,
       metadata=ToolMetadata(
           name="Taiwanese",
           description=(
               "提供台灣的原住民資料、節慶、慶典、歷史故事。 "
               "Use a detailed plain text question as input to the tool."
           ),
       ),
   ),
]

In [22]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
   query_engine_tools,
   llm=llm,
   verbose=True
)

response = agent.chat("你知道阿美族嗎？可以詳細介紹他嗎？")
print(str(response))

Thought: The current language of the user is: zh_TW. I need to use a tool to help me answer the question.
Action: Taiwanese
Action Input: {'input': '阿美族'}
Observation: 阿美族是台灣原住民族群中人口最多的一族，主要分布在中央山脈以東、立霧溪以南的平原地區，包含花蓮、台東、以及屏東縣的恆春半島[1][4]。 阿美族人的部落式民族社會在歷經衝突與調適後，逐漸與外來的國家力量取得平衡[3]。
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 阿美族是台灣原住民族群中人口最多的一個族群，主要分布在台灣中央山脈以東、立霧溪以南的平原地區，包括花蓮、台東和屏東縣的恆春半島。阿美族的部落式民族社會經過衝突和調適後，逐漸與外來的國家力量達成平衡。
阿美族是台灣原住民族群中人口最多的一個族群，主要分布在台灣中央山脈以東、立霧溪以南的平原地區，包括花蓮、台東和屏東縣的恆春半島。阿美族的部落式民族社會經過衝突和調適後，逐漸與外來的國家力量達成平衡。


In [23]:
for source in response.source_nodes:
    print(source.node.get_text())

Source 1:
阿美族是原住民族群中人口最多的一族，主要分布在中央山脈以東、立霧溪以南的平原地區，包含臺灣東部的花蓮、臺東兩縣，以及屏東縣的恆春半島。阿美族人分布遼闊，按照區域與風俗可區分為北部阿美（又稱為南勢阿美）、中部阿美（包含花東縱谷的秀姑巒阿美與海岸地區）、南部阿美（包含臺東阿美與恆春半島的阿美族）三大區塊。阿美族人與外人接觸的歷史紀錄，最早可追溯到四百年前荷蘭金礦調查隊的相關資料，不過阿美族與外界積極、密切的互動是在清朝統治臺灣的最後二十年。 清同治13 年（1874）發生牡丹社事件，為恐怕國際勢力介入原住民地區，清朝推動開山撫番政策；除開闢北、中、南通往東部的道路，並鼓勵漢人前往東部發展。

Source 2:
為了闢建道路，清朝過度徵調族人整飭道路，雙方因溝通不良而發生衝突；光緒3 年（1877）年東海岸的靜埔與港口部落聯合反抗清軍，被稱為「大港口事件」。清末也因為漢人的移住，阿美族人逐漸與漢人有比較頻繁的接觸，學習到水稻耕作技術外，亦認識到漢人的文化與風俗。 二十世紀初日本殖民統治時期，日本政府在山腳設立隘勇線，防守太魯閣族與布農族，並要求鄰近的阿美族部落協助，日方強勢的作風導致阿美族人群起反抗。日明治41 年（1908），花蓮七腳川社因為日本政府藉口隘勇脫逃、怠忽職守，欲大力建設東臺灣而遭到掃蕩，被稱為「七腳川事件」。明治44年（1911）東海岸的麻荖漏部落（Madawdaw）、都歷部落（Turik）又因被日人羞辱苛責而發動抗日，稱為「成廣澳事件」。

Source 3:
阿美族人的部落式民族社會，在歷經衝突與調適後，才逐漸與外來的國家力量取得平衡。

Source 4:
祖靈屋舉行。主要活動內容是 doloi（神石）的祭儀，祈求保佑族人平安、健康，並以 taavala 競賽活動來考驗年輕人是否有能力抵抗外侮、保護族人安全。 阿美族 簡介        阿美族人群聚而居，部落規模大、人口多，祭典活動特別盛大，以每年的豐年祭典最具代表性。 美麗的家園        阿美族自稱為「pangcah」（邦查），含有「人」、「同族人」的意思，臺東的阿美族人多數住在卑南族的北邊，被卑南族人稱為「Amis」，有北方人、北方民族的意思，後來受到學術界的採用與傳播，成為廣為人知的族群名稱。阿美族的起源神話中，有「創生神話」以及「發祥傳說」兩大類別系統；北部阿美族

In [24]:
response = agent.chat("豐年祭在什麼時候")
print(str(response))

for source in response.source_nodes:
    print(source.node.get_text())

Thought: The current language of the user is: zh_TW. I need to use a tool to help me answer the question.
Action: Taiwanese
Action Input: {'input': '豐年祭在什麼時候'}
Observation: 豐年祭通常在七、八月間舉行，因為是以農耕收成之後的日子作為祭典日，但台灣南北氣候有所差異，所以各地祭典的時間也有差別[1]。
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 豐年祭通常在七、八月間舉行，因為是以農耕收成之後的日子作為祭典日，但台灣南北氣候有所差異，所以各地祭典的時間也有差別。
豐年祭通常在七、八月間舉行，因為是以農耕收成之後的日子作為祭典日，但台灣南北氣候有所差異，所以各地祭典的時間也有差別。
Source 1:
」這些狀況在種小米時期是絕對不可以的。因為作物的改變，使得現今的豐年祭呈現較為熱鬧的場面。         豐年祭又因各地區不同而有不同的稱呼，如北部阿美族普遍稱Malalikit，東海岸阿美族則稱為Malikoda，中部阿美則稱為Ilisin。一般來說，阿美族的豐年祭是七、八月間舉行，因為是以農耕收成之後的日子作為祭典日，但台灣南北氣候有所差異，所以各地祭典的時間也有差別。豐年祭切確舉行的時間，首先要舉行部落籌備會議，由頭目召集頭目幕僚團，籌備會議的主要議題是決定活動的日期和日數。         豐年祭系列的活動中，歌舞是豐年祭活動的重心，有歌舞才是完整的豐年祭。豐年祭的歌舞男女有別，由男子擔任主要角色，女子僅居於助興地位。

Source 2:
(4)聯合年祭：(元月2日舉行)由八社輪流舉辦，有八社的表演及競賽，是觀摩卑南八社文化的最好時機。 (三) 初鹿村：以七月的收穫祭及十二月底的年祭較受重視。其中收穫祭的盪鞦韆活動是一大特色。 (四) 檳榔村：以三月八日婦女除草完工的傳統節日最具特色，顯現了檳榔村婦女團隊組織的嚴謹。十二月底的年祭亦是主要祭典之一。 (五) 利嘉村：以十二月底的大獵祭為主。 (六) 泰安村：以十二月底的大獵祭為主。 (七) 

In [25]:
response = agent.chat("台灣最高樓是什麼？")
print(str(response))

Thought: The current language of the user is: zh_TW. I need to use a tool to help me answer the question.
Action: Taiwanese
Action Input: {'input': '台灣最高樓是什麼？'}
Observation: None of the provided sources mention the tallest building in Taiwan.
Thought: I cannot answer the question with the provided tools.
Answer: 台灣最高的建築物是台北101大樓。
台灣最高的建築物是台北101大樓。


In [29]:
response = agent.chat("台灣有哪些原住民族？請你說出每一族的特色")
print(str(response))

for source in response.source_nodes:
    print(source.node.get_text())

Thought: The current language of the user is: zh-TW. I need to use a tool to help me answer the question.
Action: Taiwanese
Action Input: {'input': '台灣有哪些原住民族？請你說出每一族的特色'}
Observation: 台灣有以下原住民族：
1. 阿美族：分佈在中央山脈東側，立霧溪以南，太平洋沿岸的東台縱谷及東海岸平原，大部份居住於平地，只有極少數居住於山谷中。
2. 泰雅族：分布在台灣中北部山區，包括埔里至花蓮連線以北地區。
3. 排灣族：以台灣南部為活動區域，北起大武山地，南達恆春，西自隘寮，東到太麻里以南海岸。
4. 布農族：分布於中央山脈海拔一千至二千公尺的山區，廣及於高雄縣那瑪夏鄉、台東縣海端鄉，以南投縣境為主。
5. 卑南族：分布於台東縱谷南部。
6. 魯凱族：分布於高雄縣茂林鄉、屏東縣霧台鄉及台東縣東興村等地。
7. 鄒族：主要居住於嘉義縣阿里山鄉，亦分布於南投縣信義鄉。
8. 賽夏族：居住於新竹縣與苗栗縣交界的山區，分為北賽夏和南賽夏兩大族群 [2].

每個族群都有獨特的居住地區和特色，反映了他們對自然環境的適應和文化傳承。
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 台灣有以下原住民族：
1. 阿美族：分佈在中央山脈東側，立霧溪以南，太平洋沿岸的東台縱谷及東海岸平原，擅長拼板舟技術，以及以獨特的節奏和舞蹈表達情感。
2. 泰雅族：分布在台灣中北部山區，包括埔里至花蓮連線以北地區，以織布和紋面文化聞名。
3. 排灣族：以台灣南部為活動區域，北起大武山地，南達恆春，西自隘寮，東到太麻里以南海岸，擁有多神信仰，並且有著複雜的階級社會制度。
4. 布農族：分布於中央山脈海拔一千至二千公尺的山區，廣及於高雄縣那瑪夏鄉、台東縣海端鄉，以南投縣境為主，以八部合音聞名。
5. 卑南族：分布於台東縱谷南部，擁有豐富的農耕文化。
6. 魯凱族：分布於高雄縣茂林鄉、屏東縣霧台鄉及台東縣東興村等地，以獨特的音樂和舞蹈傳統聞名。
7. 鄒族：主要居住於嘉義縣阿里山鄉，亦分